In [ ]:
!pip install pythainlp

     |████████████████████████████████| 11.5 MB 15.4 MB/s 


In [ ]:
import nltk
import os
import re
import math
import operator
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from pythainlp.corpus import thai_stopwords
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
# Stopwords = set(stopwords.words('english'))
Stopwords=thai_stopwords()
wordlemmatizer = WordNetLemmatizer()

def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
       if word not in words_unique:
           words_unique.append(word)
    for word in words_unique:
       dict_freq[word] = words.count(word)
    return dict_freq

def tf_score(word,sentence):
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence:
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf
def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    for sentence in sentences:

        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf
def tf_idf_score(tf,idf):
    return tf*idf
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf
def sentence_importance(sentence,dict_freq,sentences):
     sentence_score = 0
     no_of_sentences = len(sentences)
    #  pos_tagged_sentence = pos_tagging(sentence)
     for word in sentence:
                sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
     return sentence_score

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def sentence_seg(test_samples_X):
    len_word=len(test_samples_X)
    test_samples_X=test_samples_X + [' ','x','x','x','x']
    sentence_list = []
    sentence = []
    idx, count, lenght, sam = 0 , 0 , 5 , 0
    for idx in range(len_word) :
        if test_samples_X[idx] == ' ' :
            for count in range(1, lenght+1) :
                if test_samples_X[idx+count] == ' ' :
                    sam = 0
                    break
                elif test_samples_X[idx+count] != ' ' :
                    sam += 1
                    if sam == lenght :
                        sentence_list.append(sentence)
                        sentence = []
                        sam = 0
                        break
        elif test_samples_X[idx] != ' ' :
            sentence.append(test_samples_X[idx])
    sentence_list.append(sentence)
    return sentence_list

In [ ]:
text=['คดี', 'นี้', 'โจทก์', 'เป็น', 'นิติบุคคล', 'ประเภท', 'บริษัท', 'จำกัด', ' ', 'ประกอบ', 'ธุรกิจ', ' ', 'จำหน่าย', 'เครื่อง', 'สำอางค์', ' ', 'ส่วน','จำเลย', 'อ้าง', 'ตัว', 'ว่า', 'เป็น', 'นายหน้า', 'ของ', ' โรง', 'งาน', 'ผลิต', 'แป้ง', 'รอง', 'พื้น', ' ', 'สามารถ', 'ว่าจ้าง', 'และ', 'สั่ง', 'ผลิต', 'แป้ง', 'รอง', 'พื้น', 'จาก', 'โรง', 'งาน', 'ได้', 'ราคา', 'ถูก', ' ', 'โจทก์', 'หลง', 'เชื่อ', 'จึง', 'ได้', 'ว่าจ้าง', 'ให้', 'จำเลย', 'ทำการ', 'ผลิต', 'เครื่อง', 'สำอางค์', ' ', '(', 'แป้ง', 'รอง', 'พื้น', ')', ' ', 'ให้', 'กับ', 'โจทก์', 'โอน', 'เงิน', 'ให้', 'กับ', 'จำเลย', 'เป็น', 'เงิน', 'จำนวน', ' ', '200,000', ' ', 'บาท', 'เป็น', 'เงิน', 'มัด', 'จำ', 'ค่า', 'ผลิต', 'แป้ง', 'รอง', 'พื้น', 'หลังจาก', 'นั้น', 'โจทก์', 'ได้', 'พยายาม', 'ติดตาม', 'ทวง', 'ถาม', 'ให้', 'จำเลย', 'ส่งมอบ', 'สินค้า', 'แต่', 'จำเลย', 'ก็', 'บ่าย', 'เบี่ยง', ' ', 'อ้าง', 'เหตุผล', 'ต่าง', 'ๆ', 'นา', 'ๆ', ' ', 'โดย', 'ระหว่าง', 'ที่', 'จำเลย', 'อ้าง', 'เหตุผล', 'ต่าง', 'ๆ', 'เพื่อ', 'บ่ายเบี่ยง', 'นั้น', ' ', 'จำเลย', 'ได้', 'ส่ง', 'เอกสาร', 'ที่', 'อ้าง', 'ว่า', 'เป็น', 'หนังสือ', 'ที่', 'บริษัท', ' ', 'ที่', 'รับ', 'ผลิต', 'ได้รับ', 'มัด', 'จำ', 'จำนวน', ' ', '200,000', ' ', 'บาท', 'ไว้', 'แล้ว', 'ให้', 'กับ', 'โจทก์', ' ', 'ต่อมา', 'โจทก์', 'เริ่ม', 'เอะใจ', 'ว่า', 'จำเลย', 'จะ', 'หลอกลวง', ' ', 'จึง', 'ได้', 'มา', 'หา', 'ผม', ' ', 'ผม', 'จึง', 'ได้', 'ให้', 'ทีม', 'งาน', 'เดินทาง', 'ไป', 'ยัง', 'บริษัท', 'ที่', 'มี', 'ชื่อ', 'ตาม', 'หนังสือ', 'ดังกล่าว', ' ', 'เพื่อ', 'สอบถาม', 'ความ', 'คืบหน้า', 'โดย', 'ตรง', 'จาก', 'บริษัท', 'ผู้', 'รับ', 'ผลิต', ' ', 'ปรากฏ', 'ว่า', 'ทาง', 'บริษัท', 'ยืนยัน', 'ว่า', ' ', 'ไม่', 'ได้รับ', 'จ้าง', 'ผลิต', 'แป้ง', 'และ', 'ไม่', 'เคย', 'ได้รับ', 'เงิน', 'จำนวน', 'ดังกล่าว', 'จาก', 'จำเลย', 'ผม', 'จึง', 'ได้', 'ขอ', 'ให้', 'ทาง', 'บริษัท', 'จึง', 'ได้', 'ทำ', 'หนังสือ', 'ยืนยัน', 'มา', 'ว่า', 'ไม่', 'เคย', 'รับ', 'จ้าง', 'ผลิต', 'แป้ง', 'รอง', 'พื้น', 'และ', 'ไม่', 'เคย', 'ได้รับ', 'เงิน', 'มัด', 'จำ', 'จำนวน', ' ', '200,000', ' ', 'บาท', ' ', 'เพื่อ', 'ใช้', 'เป็น', 'หลักฐาน', 'ใน', 'การ', 'ดำเนินคดี', ' ', 'เมื่อ', 'พิจารณา', 'จาก', 'พยาน', 'หลักฐาน', 'ทั้งหมด', 'แล้ว', ' ', 'จึง', 'แน่ชัด', 'ว่า', 'การ', 'กระทำ', 'ของ', 'จำเลย', 'จึง', 'เป็น', 'ความ', 'ผิด', 'ฐาน', 'ฉ้อโกง', ' ', 'ปลอม', 'เอกสาร', ' ', 'และ', 'ใช้', 'เอกสาร', 'ปลอม', ' ', 'ตาม', 'ประมวล', 'กฎหมาย', 'อาญา', 'มาตรา', ' ', '341', ' ', 'มาตรา', ' ', '264', ' ', 'และ', 'มาตรา', ' ', '268', ' ', 'ผม', 'จึง', 'ได้', 'ยื่น', 'ฟ้อง', 'คดี', 'ใน', 'ข้อหา', 'ดังกล่าว', 'ไป']

te=[]
for i in text:
  if i!=' ':
    te.append(i)
print(text)

word_freq = freq(te)
sentence_list=sentence_seg(text)
sents=[]
for s in sentence_list:
  sents.append([word for word in s if word not in Stopwords])
print(sents)

['คดี', 'นี้', 'โจทก์', 'เป็น', 'นิติบุคคล', 'ประเภท', 'บริษัท', 'จำกัด', ' ', 'ประกอบ', 'ธุรกิจ', ' ', 'จำหน่าย', 'เครื่อง', 'สำอางค์', ' ', 'ส่วน', 'จำเลย', 'อ้าง', 'ตัว', 'ว่า', 'เป็น', 'นายหน้า', 'ของ', ' โรง', 'งาน', 'ผลิต', 'แป้ง', 'รอง', 'พื้น', ' ', 'สามารถ', 'ว่าจ้าง', 'และ', 'สั่ง', 'ผลิต', 'แป้ง', 'รอง', 'พื้น', 'จาก', 'โรง', 'งาน', 'ได้', 'ราคา', 'ถูก', ' ', 'โจทก์', 'หลง', 'เชื่อ', 'จึง', 'ได้', 'ว่าจ้าง', 'ให้', 'จำเลย', 'ทำการ', 'ผลิต', 'เครื่อง', 'สำอางค์', ' ', '(', 'แป้ง', 'รอง', 'พื้น', ')', ' ', 'ให้', 'กับ', 'โจทก์', 'โอน', 'เงิน', 'ให้', 'กับ', 'จำเลย', 'เป็น', 'เงิน', 'จำนวน', ' ', '200,000', ' ', 'บาท', 'เป็น', 'เงิน', 'มัด', 'จำ', 'ค่า', 'ผลิต', 'แป้ง', 'รอง', 'พื้น', 'หลังจาก', 'นั้น', 'โจทก์', 'ได้', 'พยายาม', 'ติดตาม', 'ทวง', 'ถาม', 'ให้', 'จำเลย', 'ส่งมอบ', 'สินค้า', 'แต่', 'จำเลย', 'ก็', 'บ่าย', 'เบี่ยง', ' ', 'อ้าง', 'เหตุผล', 'ต่าง', 'ๆ', 'นา', 'ๆ', ' ', 'โดย', 'ระหว่าง', 'ที่', 'จำเลย', 'อ้าง', 'เหตุผล', 'ต่าง', 'ๆ', 'เพื่อ', 'บ่ายเบี่ยง', 'นั้น', ' ', 

In [ ]:
def identity_fun(text):
    return text
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer = 'word', #this is default
                                   tokenizer=identity_fun, #does no extra tokenizing
                                   preprocessor=identity_fun, #no extra preprocessor
                                   token_pattern=None)
tfidf_vector= tfidf_vectorizer.fit_transform(sents)
tfidf_array = np.array(tfidf_vector.todense())
df = pd.DataFrame(tfidf_array,columns=tfidf_vectorizer.get_feature_names())
word=[]
for i in df.columns:
  # print(i,df[i][:].sum())
  word.append((df[i][:].sum(),i))
df
# word.sort()
# print(df.apply(lambda s: s.nlargest(10).index.tolist(), axis=1).ravel()) 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,โรง,(,),"200,000",264,268,341,กฎหมาย,ข้อหา,คดี,คืบหน้า,ค่า,งาน,จำกัด,จำนวน,จำหน่าย,จำเลย,จ้าง,ฉ้อโกง,ชื่อ,ฐาน,ดำเนินคดี,ตัว,ติดตาม,ต่อมา,ถาม,ทวง,ทำ,ทำการ,ทีม,ธุรกิจ,นา,นายหน้า,นิติบุคคล,บริษัท,บาท,บ่าย,บ่ายเบี่ยง,ประมวล,ประเภท,ปลอม,ผม,ผลิต,พยาน,พิจารณา,พื้น,ฟ้อง,มัด,มาตรา,ยื่น,รอง,ราคา,ว่าจ้าง,สอบถาม,สั่ง,สำอางค์,สินค้า,ส่งมอบ,หนังสือ,หลง,หลอกลวง,หลักฐาน,หา,อาญา,อ้าง,เครื่อง,เงิน,เดินทาง,เบี่ยง,เอกสาร,เอะใจ,แน่ชัด,แป้ง,โจทก์,โรง,โอน
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.309282,0.000000,0.000000,0.000000,0.350005,0.000000,0.350005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.350005,0.000000,0.000000,0.350005,0.224184,0.000000,0.000000,0.000000,0.000000,0.350005,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.309282,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.309282,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.224184,0.000000,0.000000
1,0.400194,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.320595,0.000000,0.000000,0.000000,0.215372,0.000000,0.000000,0.000000,0.000000,0.000000,0.400194,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.400194,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.240997,0.00000,0.00000,0.274033,0.000000,0.000000,0.000000,0.000000,0.274033,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.294970,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.274033,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.321940,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.242008,0.00000,0.00000,0.275183,0.000000,0.000000,0.000000,0.000000,0.275183,0.401872,0.355115,0.000000,0.401872,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.275183,0.000000,0.401872,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.231483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.43013,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.259025,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.380085,0.000000,0.000000,0.380085,0.000000,0.000000,0.000000,0.43013,0.000000,0.000000,0.000000,0.000000,0.000000,0.380085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.275506,0.000000,0.000000
4,0.000000,0.541796,0.541796,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.370996,0.000000,0.000000,0.000000,0.000000,0.370996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.370996,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.340034,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.340034,0.000000,0.2284

In [ ]:
input_user = int(input('Percentage of information to retain(in percent):'))
no_of_sentences = int((input_user * len(sentence_list))/100)
print(no_of_sentences)
c = 1
sentence_with_importance = {}
for sent in sentence_list:
    sentenceimp = sentence_importance(sent,word_freq,sentence_list)
    sentence_with_importance[c] = sentenceimp
    c = c+1
sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)
cnt = 0
sentence_no = []
print(sentence_with_importance)
for word_prob in sentence_with_importance:
    if cnt < no_of_sentences:
        sentence_no.append(word_prob[0])
        cnt = cnt+1
    else:
      break
sentence_no.sort()
print(sentence_no)

for i in sentence_list:
  print(' '.join(i))
# print(summary,len(summary))

Percentage of information to retain(in percent):100
23
[(22, 2.0223893841188305), (18, 1.8508517779264513), (8, 1.414263787138149), (21, 1.3338315338120108), (6, 1.2295130613309142), (20, 1.189710695638175), (1, 1.1075362569580935), (17, 1.066834407482427), (13, 1.0479097097417662), (16, 1.0179449467000337), (7, 1.0080416152375729), (19, 1.000822224961901), (23, 0.9953765889760774), (9, 0.9853065882780639), (15, 0.9570348687497336), (3, 0.9454871736566054), (5, 0.9423458334159818), (12, 0.9224805448817742), (10, 0.8777525112720398), (2, 0.8688913292625112), (4, 0.8667299798018927), (11, 0.8450035484420289), (14, 0.8448615911440868)]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
คดี นี้ โจทก์ เป็น นิติบุคคล ประเภท บริษัท จำกัด ประกอบ ธุรกิจ จำหน่าย เครื่อง สำอางค์
ส่วน จำเลย อ้าง ตัว ว่า เป็น นายหน้า ของ  โรง งาน ผลิต แป้ง รอง พื้น
สามารถ ว่าจ้าง และ สั่ง ผลิต แป้ง รอง พื้น จาก โรง งาน ได้ ราคา ถูก
โจทก์ หลง เชื่อ จึง ได้ ว่าจ้าง ให้ จำเลย ทำการ ผลิ

In [ ]:
print(word_freq)

{'คดี': 2, 'นี้': 1, 'โจทก์': 6, 'เป็น': 7, 'นิติบุคคล': 1, 'ประเภท': 1, 'บริษัท': 6, 'จำกัด': 1, 'ประกอบ': 1, 'ธุรกิจ': 1, 'จำหน่าย': 1, 'เครื่อง': 2, 'สำอางค์': 2, 'ส่วน': 1, 'จำเลย': 10, 'อ้าง': 4, 'ตัว': 1, 'ว่า': 7, 'นายหน้า': 1, 'ของ': 2, ' โรง': 1, 'งาน': 3, 'ผลิต': 8, 'แป้ง': 6, 'รอง': 5, 'พื้น': 5, 'สามารถ': 1, 'ว่าจ้าง': 2, 'และ': 5, 'สั่ง': 1, 'จาก': 4, 'โรง': 1, 'ได้': 9, 'ราคา': 1, 'ถูก': 1, 'หลง': 1, 'เชื่อ': 1, 'จึง': 8, 'ให้': 7, 'ทำการ': 1, '(': 1, ')': 1, 'กับ': 3, 'โอน': 1, 'เงิน': 5, 'จำนวน': 4, '200,000': 3, 'บาท': 3, 'มัด': 3, 'จำ': 3, 'ค่า': 1, 'หลังจาก': 1, 'นั้น': 2, 'พยายาม': 1, 'ติดตาม': 1, 'ทวง': 1, 'ถาม': 1, 'ส่งมอบ': 1, 'สินค้า': 1, 'แต่': 1, 'ก็': 1, 'บ่าย': 1, 'เบี่ยง': 1, 'เหตุผล': 2, 'ต่าง': 2, 'ๆ': 3, 'นา': 1, 'โดย': 2, 'ระหว่าง': 1, 'ที่': 5, 'เพื่อ': 3, 'บ่ายเบี่ยง': 1, 'ส่ง': 1, 'เอกสาร': 3, 'หนังสือ': 3, 'รับ': 3, 'ได้รับ': 4, 'ไว้': 1, 'แล้ว': 2, 'ต่อมา': 1, 'เริ่ม': 1, 'เอะใจ': 1, 'จะ': 1, 'หลอกลวง': 1, 'มา': 2, 'หา': 1, 'ผม': 4, 'ทีม': 1, 'เดิน

In [ ]:
for i in sentence_with_importance:
  print(' '.join(sentence_list[i[0]-1]))

ตาม ประมวล กฎหมาย อาญา มาตรา 341 มาตรา 264 และ มาตรา 268
ไม่ ได้รับ จ้าง ผลิต แป้ง และ ไม่ เคย ได้รับ เงิน จำนวน ดังกล่าว จาก จำเลย ผม จึง ได้ ขอ ให้ ทาง บริษัท จึง ได้ ทำ หนังสือ ยืนยัน มา ว่า ไม่ เคย รับ จ้าง ผลิต แป้ง รอง พื้น และ ไม่ เคย ได้รับ เงิน มัด จำ จำนวน 200,000 บาท
อ้าง เหตุผล ต่าง ๆ นา ๆ
จึง แน่ชัด ว่า การ กระทำ ของ จำเลย จึง เป็น ความ ผิด ฐาน ฉ้อโกง ปลอม เอกสาร และ ใช้ เอกสาร ปลอม
ให้ กับ โจทก์ โอน เงิน ให้ กับ จำเลย เป็น เงิน จำนวน 200,000
เมื่อ พิจารณา จาก พยาน หลักฐาน ทั้งหมด แล้ว
คดี นี้ โจทก์ เป็น นิติบุคคล ประเภท บริษัท จำกัด ประกอบ ธุรกิจ จำหน่าย เครื่อง สำอางค์
ปรากฏ ว่า ทาง บริษัท ยืนยัน ว่า
ต่อมา โจทก์ เริ่ม เอะใจ ว่า จำเลย จะ หลอกลวง
เพื่อ สอบถาม ความ คืบหน้า โดย ตรง จาก บริษัท ผู้ รับ ผลิต
บาท เป็น เงิน มัด จำ ค่า ผลิต แป้ง รอง พื้น หลังจาก นั้น โจทก์ ได้ พยายาม ติดตาม ทวง ถาม ให้ จำเลย ส่งมอบ สินค้า แต่ จำเลย ก็ บ่าย เบี่ยง
เพื่อ ใช้ เป็น หลักฐาน ใน การ ดำเนินคดี
ผม จึง ได้ ยื่น ฟ้อง คดี ใน ข้อหา ดังกล่าว ไป
โดย ระหว่าง ที่ จำเลย อ้าง เหตุผล ต่าง ๆ เพื่อ บ่า

In [ ]:

cnt = 1
for sentence in sentence_list:
    if cnt in sentence_no:
       summary.append(sentence)
    cnt = cnt+1

print(summary)

['Women education is a catch all term which refers to the state of primary, secondary, tertiary and health education in girls and women.', 'There are 65 Million girls out of school across the globe; majority of them are in the developing and underdeveloped countries.', 'All the countries of the world, especially the developing and underdeveloped countries must take necessary steps to improve their condition of female education; as women can play a vital role in the nation’s development.', 'If we consider society as tree, then men are like its strong main stem which supports the tree to face the elements and women are like its roots; most important of them all.', 'The stronger the roots are the bigger and stronger the tree will be spreading its branches; sheltering and protecting the needy.', 'Women are the soul of a society; a society can well be judged by the way its women are treated.', 'An educated man goes out to make the society better, while an educated woman; whether she goes ou